<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/01_Avg%2C_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [433]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
import warnings
import re
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [434]:
df = pd.read_csv('car_prices.csv')
df.shape

(54273, 13)

In [435]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [436]:
for col in df.columns:
    print(f'{col}: {df[col].nunique()}')

id: 54273
brand: 53
model: 1827
model_year: 34
milage: 3212
fuel_type: 7
engine: 1061
transmission: 46
ext_col: 260
int_col: 124
accident: 2
clean_title: 1
price: 1481


In [437]:
def parse_engine_info(engine):
    if pd.isna(engine):
        return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan])

    hp = re.search(r'(\d+\.?\d*)HP', engine)
    liter = re.search(r'(\d+\.?\d*)L', engine)
    motor = re.search(r'(Straight|V)\s*\d*', engine)
    cylinder = re.search(r'(\d+)\s*Cylinder', engine)

    return pd.Series([
        hp.group(1) if hp else np.nan,
        liter.group(1) if liter else np.nan,
        motor.group(0) if motor else np.nan,
        cylinder.group(1) if cylinder else np.nan,
    ])

df[['hp', 'litr', 'motor', 'Cylinder']] = df['engine'].apply(parse_engine_info)

In [438]:
df = df.drop(columns=['clean_title', 'id'])

In [439]:
df.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,hp,litr,motor,Cylinder
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,11000,375.0,3.5,V6,6


In [440]:
df.isna().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0
accident,0


In [441]:
# def fill_nan_with_groupby(df, target_column, groupby_columns, agg_func='mean'):
#     fill_values = df.groupby(groupby_columns)[target_column].transform(agg_func)
#     df[target_column].fillna(fill_values, inplace=True)
#     return df

# df['hp'] = pd.to_numeric(df['hp'], errors='coerce')
# filled_df = fill_nan_with_groupby(df, 'hp', ['motor', 'Cylinder', 'litr'])

# nan_groups = filled_df[filled_df['hp'].isna()].groupby(['motor', 'Cylinder']).size()


# print(nan_groups)

# filled_df['hp'].fillna(filled_df['hp'].mean(), inplace=True)

In [442]:
df.isna().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0
accident,0


In [443]:
# def fill_nan_with_groupby(df, target_column, groupby_columns, agg_func='mean'):
#     fill_values = df.groupby(groupby_columns)[target_column].transform(agg_func)
#     df[target_column].fillna(fill_values, inplace=True)
#     return df[[target_column]]

# df['hp'] = pd.to_numeric(df['hp'], errors='coerce')
# df = fill_nan_with_groupby(df.copy(), 'hp', ['motor', 'Cylinder'])
# df['hp'].fillna(filled_df['hp'].mean(), inplace=True)
# filled_df.isna().sum()

In [444]:
# df['hp'] = pd.to_numeric(df['hp'], errors='coerce')
# fill_nan_with_groupby(df.copy(), 'hp', ['motor', 'Cylinder'])

In [445]:
# df['hp'] = pd.to_numeric(df['hp'], errors='coerce')
# df = fill_nan_with_groupby(df, 'hp', ['motor', 'Cylinder'])

In [ ]:
# mean_prices = df[df['Price'].notna()].groupby(by=['Route', 'Date_of_Journey', 'Additional_Info'])['Price'].mean().reset_index()
# df = pd.merge(df, mean_prices, on=['Route', 'Date_of_Journey', 'Additional_Info'], how='left', suffixes=('', '_mean'))
# df['Price'] = df['Price'].fillna(df['Price_mean'])
# df.drop(columns=['Price_mean'], inplace=True)

In [508]:
def fill_nan_mean(df, col, columns):
    df[col] = pd.to_numeric(col, errors='coerce')
    value = df[df[col].notna()].groupby(by=columns)[col].mean().reset_index()
    df = pd.merge(df, value, on=columns, how='left', suffixes=('', '_mean'))
    df[col] = df[col].fillna(df[col + '_mean'])
    df.drop(columns=[col + '_mean'], inplace=True)
    return df

In [510]:
df = fill_nan_mean(df.copy(), 'hp', ['motor', 'Cylinder', 'litr'])
df = fill_nan_mean(df.copy(), 'hp', ['motor', 'litr'])
df = fill_nan_mean(df.copy(), 'hp', ['brand', 'model'])
df = fill_nan_mean(df.copy(), 'hp', ['litr'])
df = fill_nan_mean(df.copy(), 'hp', ['model'])
df = fill_nan_mean(df.copy(), 'hp', ['price', 'brand'])
df = fill_nan_mean(df.copy(), 'hp', ['price'])

print(df['hp'].isna().sum())

54273


In [506]:
new_df = fill_nan_mean(df.copy(), 'litr', ['brand', 'model', 'hp'])

print(df['litr'].isna().sum())

TypeError: agg function failed [how->mean,dtype->object]

In [497]:
new_df[new_df['hp'].isna()]['model'].unique()

array(['Protege DX', 'E-Class 400E', '850 Turbo', 'Pickup Truck XE',
       'Rover Range Rover Sport 3.0 Supercharged HST',
       'Challenger SRT 392', '1500 Cheyenne Extended Cab',
       'E-Class D 2.5 Turbo', '1500 Cheyenne', '240SX Base', '300ZX Base',
       'Capri XR2', 'Challenger SRT8 392',
       'Rover Range Rover Velar R-Dynamic SE', 'Sonic LT',
       'i3 120Ah w/Range Extender', 'NX 350 F SPORT Handling',
       'F-250 LARIAT', 'Mustang Mach-E Select', 'Q5 S line Premium',
       'Arteon 2.0T SE', 'Veyron 16.4 Grand Sport'], dtype=object)

In [498]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,price,hp,litr,motor,Cylinder
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,11000,375.0,3.5,V6,6
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,8250,300.0,3.0,Straight 6,6
2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,15000,300.0,4.2,NaN,8
3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,63500,335.0,3.0,Straight 6,6
4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,7850,200.0,3.8,V6,6


In [ ]:
# mean_prices = df[df['hp'].notna()].groupby(by=['Route', 'Date_of_Journey', 'Additional_Info'])['hp'].mean().reset_index()
# df = pd.merge(df, mean_prices, on=['Route', 'Date_of_Journey', 'Additional_Info'], how='left', suffixes=('', '_mean'))
# df['hp'] = df['hp'].fillna(df['Price_mean'])
# df.drop(columns=['Price_mean'], inplace=True)

In [447]:
df[col] = pd.to_numeric(df[col], errors='coerce')

In [448]:
df[col].isna().sum()

4057

In [449]:
# mean_prices = df[df['Price'].notna()].groupby(by=['Route', 'Date_of_Journey', 'Additional_Info'])['Price'].mean().reset_index()
# df = pd.merge(df, mean_prices, on=['Route', 'Date_of_Journey', 'Additional_Info'], how='left', suffixes=('', '_mean'))
# df['Price'] = df['Price'].fillna(df['Price_mean'])
# df.drop(columns=['Price_mean'], inplace=True)

In [450]:
# df['hp'] = df['engine'].agg(lambda x: x.split('HP')[0] if "HP" in x else np.nan)
# df['hp'] = pd.to_numeric(df['hp'])

In [451]:
# df['litr'] = df['engine'].agg(lambda x: x.split('L').split(' ')[0] if "L" in x else np.nan)
# df['litr'] = pd.to_numeric(df['litr'])
# df.head()